In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [26]:
full = pd.read_csv('./impute_clean_data.csv')
y = full[['los_icu_grp']]
y_catigoricalDF = pd.get_dummies(full['los_icu_grp'],prefix='los_icu_grp')
x = full.drop(['los_icu_grp'],axis=1)

In [27]:
import torch
import torchvision
from torch.autograd import Variable
import torch.nn.functional as F

In [28]:
x_train = x.head(30000)
x_test = x.tail(12697)
y_train = y_catigoricalDF.head(30000)
y_test = y_catigoricalDF.tail(12697)

In [29]:
dtype = torch.FloatTensor
X_train = torch.from_numpy(x_train.values).type(dtype)
X_test = torch.from_numpy(x_test.values).type(dtype)
Y_train = torch.from_numpy(y_train.values).type(dtype)
Y_test = torch.from_numpy(y_test.values).type(dtype)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

torch.Size([30000, 101])
torch.Size([12697, 101])
torch.Size([30000, 5])
torch.Size([12697, 5])


In [30]:
N, D_in, H1, H2, D_out = 30000, 101, 100, 200 , 5
w1 = Variable(torch.randn(D_in,H1).type(dtype),requires_grad=True)
w2 = Variable(torch.randn(H1,H2).type(dtype),requires_grad=True)
w3 = Variable(torch.randn(H2,D_out).type(dtype),requires_grad=True)

In [31]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in,H1),
    torch.nn.ReLU(),
    torch.nn.Linear(H1,H2),
    torch.nn.ReLU(),
    torch.nn.Linear(H2,D_out),
    #torch.nn.ReLU(),
    #torch.nn.Sigmoid(),
)

#loss_fn = torch.nn.MSELoss(reduction='mean')
#loss_fn = torch.nn.L1Loss(reduction='mean')
loss_fn = torch.nn.SmoothL1Loss(reduction='mean')
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adadelta(model.parameters(),lr=learning_rate)

In [ ]:
for t in range(1000):
    y_pred = model(X_train)
    loss = loss_fn(y_pred,Y_train)
    #print(t,loss.data)
    model.zero_grad()
    loss.backward()
    for param in model.parameters():
        param.data -= learning_rate * param.grad.data
print(t,loss.item())

In [ ]:
_, Y_label_test = Y_test.max(1)
model.eval() # turn the model into evaluation mode
out = model(X_test)
_, pred = out.max(1)
num_correct = (pred == Y_label_test).sum().item()
num_correct / 12697